In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\anime\OneDrive\Desktop\Data Science Projects\Youtube Comment Analysis\youtube_comment_analyser\data\processed\reddit_preprocessing.csv")
df

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1
...,...,...
36788,jesus,0
36789,kya bhai pure saal chutiya banaya modi aur jab...,1
36790,downvote karna tha par upvote hogaya,0
36791,haha nice,1


In [2]:
df =  df.dropna(subset=["clean_comment"])
df.shape

(36661, 2)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import os

c:\Users\anime\OneDrive\Desktop\Data Science Projects\Youtube Comment Analysis\comment_analysis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
mlflow.set_experiment("ML Algos with Revised HP Tuning")

<Experiment: artifact_location='s3://datascienceanimesh/516785590309530379', creation_time=1743657286448, experiment_id='516785590309530379', last_update_time=1743657286448, lifecycle_stage='active', name='ML Algos with Revised HP Tuning', tags={}>

In [5]:
import mlflow
import optuna
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.under_sampling import RandomUnderSampler

# Step 1: Remove rows where 'category' is NaN
df = df.dropna(subset=['category'])

# Step 2: Train-test split BEFORE TF-IDF vectorization
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# Step 3: TF-IDF vectorization applied only on training data
ngram_range = (1, 2)  # Use bigram features
max_features = 1000  # Limit vocabulary size

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train = vectorizer.fit_transform(X_train_raw)  # Fit on training data
X_test = vectorizer.transform(X_test_raw)  # Transform test data separately

# Step 4: Apply RandomUnderSampler to balance classes in the training set
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_UnderSampler_TFIDF_Bigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Random Forest using cross-validation
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Number of trees in the forest
    max_depth = trial.suggest_int('max_depth', 3, 20)  # Maximum depth of the tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Minimum samples required to split a node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)  # Minimum samples required at a leaf node

    # RandomForestClassifier setup
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   random_state=42)

    # Use cross-validation on resampled training data
    cv_scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
    return np.mean(cv_scores)  # Return mean accuracy of cross-validation


# Step 7: Run Optuna for Random Forest, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                        max_depth=best_params['max_depth'],
                                        min_samples_split=best_params['min_samples_split'],
                                        min_samples_leaf=best_params['min_samples_leaf'],
                                        random_state=42)

    # Train the best model on the resampled training set
    best_model.fit(X_train_resampled, y_train_resampled)

    # Log the best model using MLflow
    log_mlflow("RandomForest", best_model, X_train_resampled, X_test, y_train_resampled, y_test)

# Run the experiment for Random Forest
run_optuna_experiment()


[I 2025-04-03 11:24:19,984] A new study created in memory with name: no-name-09252d46-fcb3-4472-96b9-b4d5b5859a0d
[I 2025-04-03 11:24:23,140] Trial 0 finished with value: 0.6568658333568395 and parameters: {'n_estimators': 98, 'max_depth': 13, 'min_samples_split': 19, 'min_samples_leaf': 17}. Best is trial 0 with value: 0.6568658333568395.
[I 2025-04-03 11:24:29,677] Trial 1 finished with value: 0.6648478894520273 and parameters: {'n_estimators': 171, 'max_depth': 16, 'min_samples_split': 8, 'min_samples_leaf': 15}. Best is trial 1 with value: 0.6648478894520273.
[I 2025-04-03 11:24:43,624] Trial 2 finished with value: 0.6727795936647759 and parameters: {'n_estimators': 248, 'max_depth': 18, 'min_samples_split': 4, 'min_samples_leaf': 7}. Best is trial 2 with value: 0.6727795936647759.
[I 2025-04-03 11:24:48,804] Trial 3 finished with value: 0.6313027506167627 and parameters: {'n_estimators': 229, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 18}. Best is trial 2 with val

🏃 View run RandomForest_UnderSampler_TFIDF_Bigrams at: http://ec2-16-171-237-43.eu-north-1.compute.amazonaws.com:5000/#/experiments/516785590309530379/runs/50e43e5274fb4fc3af898d9eaec94385
🧪 View experiment at: http://ec2-16-171-237-43.eu-north-1.compute.amazonaws.com:5000/#/experiments/516785590309530379
